In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Cleaned_Data_Merchant_Level.csv')
df

,Trx_Rank,Points,Trx_Vlu,Trx_Age,Customer_Age,Category In English,User_Id,Mer_Id
0,1,20140,2014.0,362,362,Fashion,21053,0
1,1,11200,1120.0,371,368,Fashion,26472,0
2,2,8500,850.0,368,368,Fashion,26472,0
3,1,1980,198.0,558,34,Fashion,27075,0
4,1,2400,240.0,413,413,Fashion,27417,0
...,...,...,...,...,...,...,...,...
43667,1,3050,305.0,27,27,F&B,26865,55
43668,1,3120,312.0,31,31,F&B,27128,55
43669,1,1200,120.0,25,25,F&B,28814,55
43670,1,20,2.0,2,2,Other,29107,56


In [45]:
df.info() # Check data types and missing values
df.describe().T # Check summary statistics

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43672 entries, 0 to 43671
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Trx_Rank             43672 non-null  int64  
 1   Points               43672 non-null  int64  
 2   Trx_Vlu              43672 non-null  float64
 3   Trx_Age              43672 non-null  int64  
 4   Customer_Age         43672 non-null  int64  
 5   Category In English  43672 non-null  object 
 6   User_Id              43672 non-null  int64  
 7   Mer_Id               43672 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 2.7+ MB


,count,mean,std,min,25%,50%,75%,max
Trx_Rank,43672.0,1.265227,0.776549,1.0,1.00,1.00,1.00,17.0
Points,43672.0,3312.249748,5756.374137,20.0,1220.00,1893.50,3100.00,263800.0
Trx_Vlu,43672.0,331.224975,575.637414,2.0,122.00,189.35,310.00,26380.0
Trx_Age,43672.0,185.186664,121.989540,1.0,62.00,203.00,271.00,698.0
Customer_Age,43672.0,144.027844,114.764448,1.0,34.00,115.00,252.00,682.0
User_Id,43672.0,17173.411889,9665.399084,0.0,8710.75,17561.00,25634.25,33518.0
Mer_Id,43672.0,16.277111,16.099288,0.0,8.00,8.00,26.00,56.0


In [46]:
df.duplicated().sum() # Check duplicate
df.dropna(inplace=True)

In [47]:
# Recency: Calculate the number of days since the last transaction for each user
recency = df.groupby('User_Id')['Trx_Age'].min()

# Frequency: Calculate the number of transactions for each user with a specific merchant
frequency = df.groupby('User_Id')['Trx_Rank'].sum()

# Monetary: Calculate the total equivalent value in pounds (GBP) for the redeemed points in each transaction
monetary = df.groupby('User_Id')['Trx_Vlu'].sum()

# Combine the RFM values into a single DataFrame
rfm_df = pd.concat([recency, frequency, monetary], axis=1)
rfm_df.columns = ['Recency', 'Frequency', 'Monetary']

rfm_df

,Recency,Frequency,Monetary
User_Id,,,
0,281,1,442.7
1,34,1,249.0
2,236,1,300.0
3,320,1,162.0
4,407,1,70.0
...,...,...,...
33514,101,6,594.0
33515,244,1,48.0
33516,256,1,132.0


In [48]:
rfm_df.isnull().sum()

Recency      0
Frequency    0
Monetary     0
dtype: int64

In [49]:
wcss = []
for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(rfm_df)
    wcss.append(kmeans.inertia_)

# Plot the Elbow curve
import matplotlib.pyplot as plt
plt.plot(range(2, 11), wcss)
plt.xlabel('Number of Clusters')
plt.ylabel('Within-cluster Sum of Squares')
plt.title('Elbow Method')
plt.show()

c:\Users\gemy2\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'split'